In [109]:
## Importação das Bibliotecas utilizadas neste Fonte
import requests
import json 
import pandas as pd

In [110]:
#Baixa realizadas Pacientes
rel_pac_baixa = requests.get('http://intranethspm:5009/hspmsgh-api/relatorio/baixapacientes/2024-04-28/2024-04-28')

In [111]:
# Verificação da resposta da requisição
rel_pac_baixa

<Response [200]>

In [112]:
# Carregando os dados 'json' para um tipo dicionário

#Ficha de Baixa do Paciente no PS
response_baixa_pac = json.loads(rel_pac_baixa.text)

In [113]:
#Fichas de Baixa do Paciente no PS no formato do DataFrame
df_pac_baixa =  pd.DataFrame.from_dict(response_baixa_pac )

In [114]:
#Profissionais
profissionais = requests.get('http://intranethspm:5009/hspmsgh-api/relatorio/profissionais/2024-04-28/2024-04-28')

In [115]:
# Carregando os dados 'json' para um tipo dicionário

#Fichas da lista dos Profissionais 
profissionais = json.loads(profissionais.text)

In [116]:
#Fichas do Profissionais no PS
df_profissionais =  pd.DataFrame.from_dict(profissionais )

In [117]:
# Join realizado nos dataframes de Baixa do Paciente e Profissionais
dataframe = pd.merge(df_pac_baixa, df_profissionais, how="inner", on=["usuario", "usuario"])

dataframe['data_mvto'] = pd.to_datetime(dataframe['data_mvto'],format='mixed') #if not already as datetime object

# Specify the starting datetime
start_datetime = pd.to_datetime('2024-04-28 07:00:00')

# Filter the DataFrame
dataframe = dataframe[dataframe['data_mvto'] >= start_datetime]
dataframe

,prontuario,con_numero,trg_seq,seqp,data_consulta,cod_status,status_ficha,usuario,data_mvto,rf_profissional,nome_profissional
0,11148483,5256514,181404,7,2024-04-28 23:28:37.363,6,ALTA MEDICA,R937746,2024-04-28 23:39:45.695,9377468,GUSTAVO NOE DE MARCO
1,11148483,5256490,181404,7,2024-04-28 21:51:04.805,6,ALTA MEDICA,R937746,2024-04-28 23:39:45.695,9377468,GUSTAVO NOE DE MARCO
2,11551447,5256488,181350,9,2024-04-28 21:44:08.096,6,ALTA MEDICA,R937746,2024-04-28 23:30:48.099,9377468,GUSTAVO NOE DE MARCO
3,11581139,5256460,181374,5,2024-04-28 19:34:16.549,6,ALTA MEDICA,R937746,2024-04-28 20:46:56.892,9377468,GUSTAVO NOE DE MARCO
4,5078340,5256452,181364,5,2024-04-28 18:52:05.922,6,ALTA MEDICA,R937746,2024-04-28 20:28:39.416,9377468,GUSTAVO NOE DE MARCO
...,...,...,...,...,...,...,...,...,...,...,...
172,12176988,5256236,181156,5,2024-04-28 06:29:11.723,6,ALTA MEDICA,H853141,2024-04-28 07:39:51.012,8531412,MARCIUS VINICIUS GONCALVES CORREIA
173,12261855,5256232,181152,6,2024-04-28 06:08:40.043,6,ALTA MEDICA,H853141,2024-04-28 12:25:04.340,8531412,MARCIUS VINICIUS GONCALVES CORREIA
174,11356805,5256286,181204,6,2024-04-28 09:59:53.068,6,ALTA MEDICA,R938219,2024-04-28 14:37:15.050,9382194,FELIPE MATOS CAMARGO
175,3937422,5256254,181174,6,2024-04-28 08:01:28.271,22,ALTA COM ENCAMINHAMENTO,H574177,2024-04-28 08:59:59.214,5741777,NEUZA ALICE NOBUKO SAWADA CHIMABUCURO


In [118]:
dataframe['data_consulta'] = pd.to_datetime(dataframe['data_consulta'],format='mixed') #if not already as datetime object

In [119]:
dataframe['data_mvto'] = pd.to_datetime(dataframe['data_mvto'],format='mixed') #if not already as datetime object

In [120]:
result = (dataframe.sort_values('data_mvto').groupby('nome_profissional')
         ['data_mvto'].agg(['first', 'last'])
       )

In [121]:
result  = result.reset_index()
result

,nome_profissional,first,last
0,EDUARDO DE FREITAS GUIMARAES,2024-04-28 13:05:34.216,2024-04-28 18:25:45.059
1,FELIPE MATOS CAMARGO,2024-04-28 14:37:15.050,2024-04-28 14:37:15.050
2,FILIPE AUGUSTO DA SILVA,2024-04-28 19:49:40.788,2024-04-28 22:22:15.977
3,GUSTAVO NOE DE MARCO,2024-04-28 20:01:57.017,2024-04-28 23:39:45.695
4,JULIANA RODRIGUES MARTINS,2024-04-28 19:31:14.992,2024-04-28 23:55:38.285
5,MARCIUS VINICIUS GONCALVES CORREIA,2024-04-28 07:39:51.012,2024-04-28 13:02:30.367
6,MARIA EDUARDA LIMA BARBALHO DE MELLO,2024-04-28 07:47:05.113,2024-04-28 07:47:05.113
7,MATHEUS EIJI ASAM MAEHATA,2024-04-28 13:33:32.280,2024-04-28 21:54:49.762
8,NEUZA ALICE NOBUKO SAWADA CHIMABUCURO,2024-04-28 08:59:59.214,2024-04-28 08:59:59.214
9,PAULO AUGUSTO TALARICO GALENO CAVALCANTI,2024-04-28 11:00:14.562,2024-04-28 20:57:36.397


In [122]:
result_2 = dataframe.groupby(['nome_profissional']).count().usuario

In [123]:
result_2

nome_profissional
EDUARDO DE FREITAS GUIMARAES                25
FELIPE MATOS CAMARGO                         1
FILIPE AUGUSTO DA SILVA                      9
GUSTAVO NOE DE MARCO                         8
JULIANA RODRIGUES MARTINS                   16
MARCIUS VINICIUS GONCALVES CORREIA           8
MARIA EDUARDA LIMA BARBALHO DE MELLO         1
MATHEUS EIJI ASAM MAEHATA                   55
NEUZA ALICE NOBUKO SAWADA CHIMABUCURO        1
PAULO AUGUSTO TALARICO GALENO CAVALCANTI     9
ROSSANO MARCELLO PEREIRA GOMES              11
VINICIUS RAFAEL VEIGA SALAZAR               18
Name: usuario, dtype: int64

In [124]:
result_3 = dataframe.groupby([ 'nome_profissional',pd.Grouper(key='data_mvto', freq='12h', origin='2024-04-29 07:00:00')]).usuario.count().to_frame()

result_reset = result_3.reset_index()
print(result_reset)
type(result_reset)

                           nome_profissional           data_mvto  usuario
0               EDUARDO DE FREITAS GUIMARAES 2024-04-28 07:00:00       25
1                       FELIPE MATOS CAMARGO 2024-04-28 07:00:00        1
2                    FILIPE AUGUSTO DA SILVA 2024-04-28 19:00:00        9
3                       GUSTAVO NOE DE MARCO 2024-04-28 19:00:00        8
4                  JULIANA RODRIGUES MARTINS 2024-04-28 19:00:00       16
5         MARCIUS VINICIUS GONCALVES CORREIA 2024-04-28 07:00:00        8
6       MARIA EDUARDA LIMA BARBALHO DE MELLO 2024-04-28 07:00:00        1
7                  MATHEUS EIJI ASAM MAEHATA 2024-04-28 07:00:00       41
8                  MATHEUS EIJI ASAM MAEHATA 2024-04-28 19:00:00       14
9      NEUZA ALICE NOBUKO SAWADA CHIMABUCURO 2024-04-28 07:00:00        1
10  PAULO AUGUSTO TALARICO GALENO CAVALCANTI 2024-04-28 07:00:00        6
11  PAULO AUGUSTO TALARICO GALENO CAVALCANTI 2024-04-28 19:00:00        3
12            ROSSANO MARCELLO PEREIRA

pandas.core.frame.DataFrame

In [125]:
# Convert 'date' column to datetime
result_reset['data_mvto'] = pd.to_datetime(result_reset['data_mvto'])

# Specify the date you're interested in
specified_date = '2024-04-28 07:00:00'

In [126]:
# Filter the DataFrame for the specified date
filtered_df = result_reset[result_reset['data_mvto'] == pd.to_datetime(specified_date)]

In [127]:
filtered_df

,nome_profissional,data_mvto,usuario
0,EDUARDO DE FREITAS GUIMARAES,2024-04-28 07:00:00,25
1,FELIPE MATOS CAMARGO,2024-04-28 07:00:00,1
5,MARCIUS VINICIUS GONCALVES CORREIA,2024-04-28 07:00:00,8
6,MARIA EDUARDA LIMA BARBALHO DE MELLO,2024-04-28 07:00:00,1
7,MATHEUS EIJI ASAM MAEHATA,2024-04-28 07:00:00,41
9,NEUZA ALICE NOBUKO SAWADA CHIMABUCURO,2024-04-28 07:00:00,1
10,PAULO AUGUSTO TALARICO GALENO CAVALCANTI,2024-04-28 07:00:00,6
12,ROSSANO MARCELLO PEREIRA GOMES,2024-04-28 07:00:00,11
13,VINICIUS RAFAEL VEIGA SALAZAR,2024-04-28 07:00:00,18


In [128]:
dataframe_1 = pd.merge(result, filtered_df, how="inner", on=["nome_profissional", "nome_profissional"])


In [129]:
dataframe_1

,nome_profissional,first,last,data_mvto,usuario
0,EDUARDO DE FREITAS GUIMARAES,2024-04-28 13:05:34.216,2024-04-28 18:25:45.059,2024-04-28 07:00:00,25
1,FELIPE MATOS CAMARGO,2024-04-28 14:37:15.050,2024-04-28 14:37:15.050,2024-04-28 07:00:00,1
2,MARCIUS VINICIUS GONCALVES CORREIA,2024-04-28 07:39:51.012,2024-04-28 13:02:30.367,2024-04-28 07:00:00,8
3,MARIA EDUARDA LIMA BARBALHO DE MELLO,2024-04-28 07:47:05.113,2024-04-28 07:47:05.113,2024-04-28 07:00:00,1
4,MATHEUS EIJI ASAM MAEHATA,2024-04-28 13:33:32.280,2024-04-28 21:54:49.762,2024-04-28 07:00:00,41
5,NEUZA ALICE NOBUKO SAWADA CHIMABUCURO,2024-04-28 08:59:59.214,2024-04-28 08:59:59.214,2024-04-28 07:00:00,1
6,PAULO AUGUSTO TALARICO GALENO CAVALCANTI,2024-04-28 11:00:14.562,2024-04-28 20:57:36.397,2024-04-28 07:00:00,6
7,ROSSANO MARCELLO PEREIRA GOMES,2024-04-28 07:28:45.628,2024-04-28 12:25:14.279,2024-04-28 07:00:00,11
8,VINICIUS RAFAEL VEIGA SALAZAR,2024-04-28 09:06:44.112,2024-04-28 18:39:34.263,2024-04-28 07:00:00,18


In [130]:
dataframe_1.rename(columns={'usuario': 'NumeroDeConsultas'}, inplace=True)

In [131]:
dataframe_1

,nome_profissional,first,last,data_mvto,NumeroDeConsultas
0,EDUARDO DE FREITAS GUIMARAES,2024-04-28 13:05:34.216,2024-04-28 18:25:45.059,2024-04-28 07:00:00,25
1,FELIPE MATOS CAMARGO,2024-04-28 14:37:15.050,2024-04-28 14:37:15.050,2024-04-28 07:00:00,1
2,MARCIUS VINICIUS GONCALVES CORREIA,2024-04-28 07:39:51.012,2024-04-28 13:02:30.367,2024-04-28 07:00:00,8
3,MARIA EDUARDA LIMA BARBALHO DE MELLO,2024-04-28 07:47:05.113,2024-04-28 07:47:05.113,2024-04-28 07:00:00,1
4,MATHEUS EIJI ASAM MAEHATA,2024-04-28 13:33:32.280,2024-04-28 21:54:49.762,2024-04-28 07:00:00,41
5,NEUZA ALICE NOBUKO SAWADA CHIMABUCURO,2024-04-28 08:59:59.214,2024-04-28 08:59:59.214,2024-04-28 07:00:00,1
6,PAULO AUGUSTO TALARICO GALENO CAVALCANTI,2024-04-28 11:00:14.562,2024-04-28 20:57:36.397,2024-04-28 07:00:00,6
7,ROSSANO MARCELLO PEREIRA GOMES,2024-04-28 07:28:45.628,2024-04-28 12:25:14.279,2024-04-28 07:00:00,11
8,VINICIUS RAFAEL VEIGA SALAZAR,2024-04-28 09:06:44.112,2024-04-28 18:39:34.263,2024-04-28 07:00:00,18


In [132]:
dataframe_1.rename(columns={'nome_profissional': 'Medico'}, inplace=True)
dataframe_1.rename(columns={'first': 'PrimeiroAtendimento'}, inplace=True)
dataframe_1.rename(columns={'last': 'UltimoAtendimento'}, inplace=True)

In [133]:
dataframe_1 = dataframe_1[['Medico','PrimeiroAtendimento','UltimoAtendimento','NumeroDeConsultas']]

In [134]:
dataframe_1

,Medico,PrimeiroAtendimento,UltimoAtendimento,NumeroDeConsultas
0,EDUARDO DE FREITAS GUIMARAES,2024-04-28 13:05:34.216,2024-04-28 18:25:45.059,25
1,FELIPE MATOS CAMARGO,2024-04-28 14:37:15.050,2024-04-28 14:37:15.050,1
2,MARCIUS VINICIUS GONCALVES CORREIA,2024-04-28 07:39:51.012,2024-04-28 13:02:30.367,8
3,MARIA EDUARDA LIMA BARBALHO DE MELLO,2024-04-28 07:47:05.113,2024-04-28 07:47:05.113,1
4,MATHEUS EIJI ASAM MAEHATA,2024-04-28 13:33:32.280,2024-04-28 21:54:49.762,41
5,NEUZA ALICE NOBUKO SAWADA CHIMABUCURO,2024-04-28 08:59:59.214,2024-04-28 08:59:59.214,1
6,PAULO AUGUSTO TALARICO GALENO CAVALCANTI,2024-04-28 11:00:14.562,2024-04-28 20:57:36.397,6
7,ROSSANO MARCELLO PEREIRA GOMES,2024-04-28 07:28:45.628,2024-04-28 12:25:14.279,11
8,VINICIUS RAFAEL VEIGA SALAZAR,2024-04-28 09:06:44.112,2024-04-28 18:39:34.263,18


In [135]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as 'dd_MM_yyyy'
formatted_date = now.strftime('%d_%m_%Y')

print(formatted_date)
# Export to Excel
dataframe_1.to_excel(r'G:\OneDrive - Default Directory\OneDrive - rede.sp\WorkSpace_PS\Relatorio_PS\Relatorio_Produtividade_28_04_2024.xlsx', index=False)


29_04_2024
